# Install Dependencies

In [2]:
# 1. Install the necessary libraries (gtts is added)
!pip install gradio deep-translator gtts

# Import Libraries

In [ ]:
# 2. Import the libraries (gTTS and tempfile are added)
import gradio as gr
from deep_translator import GoogleTranslator, exceptions
from gtts import gTTS
import tempfile

In [ ]:
# 3. Define the list of supported languages for the dropdown menus
# This section remains unchanged.
supported_languages = {
    "Auto-Detect": "auto",
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Japanese": "ja",
    "Korean": "ko",
    "Russian": "ru",
    "Chinese (Simplified)": "zh-CN",
    "Arabic": "ar",
    "Hindi": "hi",
    "Portuguese": "pt"
}

In [ ]:
# 4. NEW: Define a function to generate speech from text
def generate_speech(text, lang_code):
    """
    Generates speech from text using gTTS and returns the path to a temporary audio file.
    """
    if not text or lang_code == 'auto':
        return None  # No audio if there is no text or language is not specific

    try:
        # Create gTTS object
        tts = gTTS(text=text, lang=lang_code)

        # Save the audio to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
            tts.save(fp.name)
            return fp.name  # Return the file path

    except Exception as e:
        print(f"An error occurred in gTTS: {e}")
        return None

In [ ]:
# 5. MODIFIED: Define the core translation function to include TTS output
def translate_text(text, source_lang, target_lang):
    """
    Translates text and generates speech, with error handling.
    """
    if not text:
        return "", "", None  # Return None for the audio path

    source_code = supported_languages[source_lang]
    target_code = supported_languages[target_lang]

    try:
        # Perform the translation
        translator = GoogleTranslator(source=source_code, target=target_code)
        translated_text = translator.translate(text)

        # Detect the source language if 'auto-detect' was selected
        if source_code == 'auto':
            detected_source_lang_code = GoogleTranslator().detect(text)[0]
            detected_source_lang_name = [name for name, code in supported_languages.items() if code == detected_source_lang_code]
            detected_lang_str = f"Detected Language: {detected_source_lang_name[0] if detected_source_lang_name else detected_source_lang_code}"
        else:
            detected_lang_str = f"Source: {source_lang}"

        # Generate speech from the translated text
        audio_filepath = generate_speech(translated_text, target_code)

        return translated_text, detected_lang_str, audio_filepath  # Return the audio file path

    except exceptions.NotValidPayload as e:
        return f"Error: The text may be too short or empty. Details: {e}", "", None
    except Exception as e:
        return f"An unexpected error occurred: {e}", "", None

In [ ]:
# 6. Define a function to swap languages and text
# This section remains unchanged.
def swap_inputs(text, lang1, lang2, translated_text):
    if lang1 == "Auto-Detect":
      return text, lang1, lang2, translated_text

    new_source_text = translated_text
    new_translated_text = text

    return new_source_text, lang2, lang1, new_translated_text

# 7. MODIFIED: Build the interactive interface with Gradio Blocks
with gr.Blocks(theme=gr.themes.Soft(), css=".gradio-container {background-color: #f5f5f5}") as demo:
    gr.Markdown(
        """
        # 🌐 Real-Time Language Translator with Audio
        Enter text, select the languages, see the translation, and listen to it!
        """
    )

    with gr.Row(variant="panel"):
        # Input Column (Unchanged)
        with gr.Column(scale=2):
            source_text = gr.Textbox(lines=7, label="Text to Translate", placeholder="Type or paste text here...")

            with gr.Row():
                source_lang = gr.Dropdown(
                    choices=list(supported_languages.keys()),
                    value="Auto-Detect",
                    label="Source Language"
                )
                swap_button = gr.Button(value="🔄 Swap", variant="secondary")
                target_lang = gr.Dropdown(
                    choices=[lang for lang in supported_languages.keys() if lang != "Auto-Detect"],
                    value="Spanish",
                    label="Target Language"
                )

            translate_button = gr.Button("Translate", variant="primary")
            clear_button = gr.ClearButton()

        # MODIFIED: Output Column
        with gr.Column(scale=2):
            translated_text = gr.Textbox(lines=7, label="Translated Text", interactive=False, placeholder="Translation will appear here...")
            # NEW: Audio player component
            tts_output = gr.Audio(label="Listen to Translation", autoplay=False)
            detected_language_output = gr.Textbox(label="Detection Info", interactive=False)

    gr.Examples(
        examples=[
            ["Hello, how are you today?", "English", "French"],
            ["Me encanta programar en Python.", "Spanish", "English"],
            ["Wie geht es Ihnen?", "German", "English"]
        ],
        inputs=[source_text, source_lang, target_lang]
    )

    # MODIFIED: Define interactions
    translate_button.click(
        fn=translate_text,
        inputs=[source_text, source_lang, target_lang],
        # Add tts_output to the list of outputs
        outputs=[translated_text, detected_language_output, tts_output]
    )

    swap_button.click(
        fn=swap_inputs,
        inputs=[source_text, source_lang, target_lang, translated_text],
        outputs=[source_text, source_lang, target_lang, translated_text]
    )

    # MODIFIED: Connect the clear button to the new audio component
    clear_button.add([source_text, translated_text, detected_language_output, tts_output])

In [ ]:
# 8. Launch the app (use debug=True for Colab)
demo.launch(debug=True , share = True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://437ec8648f16461da5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
